In [ ]:
%pip install langchain
%pip install openai
%pip install python-dotenv

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
]

In [14]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.environ.get("OPENAI_API_KEY")

os.environ["OPENAI_PROXY"] = "http://localhost:7890"

llm = OpenAI()

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

In [ ]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [ ]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

# This chain uses an LLM to determine how to route things.
router_chain = LLMRouterChain.from_llm(llm, router_prompt) 

In [ ]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [15]:
print(chain.run("What is black body radiation?"))



> Entering new  chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


Black body radiation is the thermal electromagnetic radiation emitted by a black body in thermodynamic equilibrium. It is an idealized physical body that absorbs all electromagnetic radiation that falls on it, regardless of frequency or angle of incidence. The radiation is described by the Planck radiation law, which quantifies the spectral radiance of a black body as a function of its temperature.


In [16]:
print(
    chain.run(
        "What is the first prime number greater than 40 such that one plus the prime number is divisible by 3"
    )
)



> Entering new  chain...
math: {'input': 'What is the first prime number greater than 40 such that one plus the prime number is divisible by 3'}
> Finished chain.
?

The first prime number greater than 40 such that one plus the prime number is divisible by 3 is 43.


In [17]:
print(chain.run("What is the name of the type of cloud that rains"))



> Entering new  chain...
physics: {'input': 'What is the name of the type of cloud that rains?'}
> Finished chain.


The name of the type of cloud that rains is a cumulonimbus cloud.
